In [1]:
#!pip install jupyter_ui_pol

In [2]:
from IPython. display import display, HTML, Image, clear_output
import time
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import random
import json 

In [3]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(interval=0.001, max_rate=20, allow_interupt=True):    
    
    start_wait = time.time()
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)# what does interval do here?
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return
    

In [4]:
button_options= ["a", "b", "c",">9","7-9","<7", "female", "male", "non-binary", "prefer no to say"]
btn1= widgets.Button(description= button_options[0])
btn2= widgets.Button(description= button_options[1]) 
btn3= widgets.Button(description= button_options[2]) 
btn4= widgets.Button(description= button_options[3])
btn5= widgets.Button(description= button_options[4]) 
btn6= widgets.Button(description= button_options[5]) 
btn7= widgets.Button(description= button_options[6])
btn8= widgets.Button(description= button_options[7])
btn9= widgets.Button(description= button_options[8])
btn10= widgets. Button(description= button_options[9])
btn1.on_click(register_event) 
btn2.on_click(register_event)
btn3.on_click(register_event)
btn4.on_click(register_event)
btn5.on_click(register_event)
btn6.on_click(register_event)
btn7.on_click(register_event)
btn8.on_click(register_event)
btn9.on_click(register_event)
btn10.on_click(register_event)


In [5]:
def send_to_google_form(data_dict, form_url):
    
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


In [6]:
def display_grid(image, clear=False):
    img = Image(image, width = 300)
    display(img)
    if clear == True:
        time.sleep(20)
        clear_output()
        time.sleep(0.5)
    else:
        time.sleep(0.1)

In [7]:
sentences = ["Welcome to a memory test for you to showoff your big brain:3",
                 "Enter your ID:",
                 "Please enter the hours of sleep you had last night:",
                 "Please indicate your gender:",
                 "In this test, you will be given a few graphs with different shapes in various colors",
                 "You will need to memorize it as much as possible to answer the following multiple choice questions",
                 "You will be given 20 seconds each time the graph appears",
                 "Be prepared for increasingly complex graphs as you progress",
                 "The total time you used and your score will be displayed at the end of the test:)",
                 "Now the test begins in:",
                 "3",
                 "2",
                 "1"]
durations = [4, 1, 1, 1, 4, 5, 4, 4, 4, 2, 1, 1, 1]
#durations = [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]

id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifier would be CBTC
"""

def display_intro(sentences, durations):
    for idx in range(len(sentences)):
        sentence = sentences[idx]
        duration = durations[idx]
        html_output = HTML(f"<h2>{sentence}</h2>")
        display(html_output)
        if idx==1:
            print(id_instructions)
            ID= input("")
            data_dict["ID"] = ID
            clear_output(wait = True)
            html_output = HTML(f"<h2>Hi,{ID}!</h2>")
            display(html_output)
        elif idx==2:
            display(widgets.HBox([btn4, btn5, btn6]))
            wait_for_event()
            hour_sleep = event_info["description"]
            data_dict["hour_sleep"] = hour_sleep
            clear_output(wait = True)
        elif idx==3:
            display(widgets.HBox([btn7, btn8, btn9, btn10]))
            wait_for_event()
            gender = event_info["description"]
            data_dict["gender"] = gender
            clear_output(wait = True)
        else:
            time.sleep(duration)
            clear_output(wait=True)


In [8]:
q1 = {"What color was the circle?(a) orange (b) blue (c) pink": "a",
      "What color was the triangle?(a) green (b) blue (c) orange": "b",
      "Which shape was yellow? (a) rectangle (b) trapezoid (c) sqaure": "b",
      "Which shape was green? (a) square  (b) pentagon (c) circle": "a"}

q2 = {"what shape was in position A? (a) circle (b) rectangle (c) sqaure": "c",
     "what shape was in position B? (a) triangle (b) trapezoid (c) hexagon": "b",
     "what shape was in position C? (a) circle (b) triangle (c) sqaure": "b"}

q3 = {"What color was the circle?(a) red (b) blue (c) pink": "a",
      "What color was the oval?(a) yellow (b) purple (c) pink": "b",
      "What color was the triangle?(a) green (b) blue (c) orange": "a",
      "What color was the diamond?(a) red (b) green (c) purple": "c",
     "Which shape was pink? (a) oval (b) star (c) sqaure": "b",
      "Which shape was orange? (a) circle  (b) pentagon (c) rectangle": "b"}


q4 = {"what shape was in position A? (a) star (b) rectangle (c) sqaure": "a",
     "what shape was in position B? (a) circle (b) hexagon (c) oval": "b",
     "what shape was in position C? (a) pentagon (b) diamond (c) circle": "c"}

q5 = {"What color was the oval?(a) yellow (b) blue (c) green": "a",
      "What color was the circle?(a) yellow (b) purple (c) pink": "b",
      "What color was the heart?(a) red (b) green (c) purple": "b",
     "Which shape was pink? (a) triangle (b) star (c) diamond": "c",
      "Which shape was black? (a) sun  (b) parallelogram (c) plus": "b",
      "Which shape was white? (a) triangle  (b) trapezoid (c) lightning": "c",
     "What shape was below the heart? (a) circle (b) pentagon (c) sqaure": "b",
     "How many shapes maximum are in the same color? (a)2 (b)3 (c)4": "a"}

q6 = {"what shape was in position A? (a) oval (b) rectangle (c) plus": "c",
     "what shape was in position B? (a) triangle (b) heart (c) hexagon": "b",
     "what shape was in position C? (a) circle (b) arrow (c) sqaure": "b",
      "what shape was in position D? (a) lightning (b) diamond (c) sun": "c",
      "what shape was in position E? (a) triangle (b) hexagon (c) pentagon": "b"}


In [9]:
def analyze_graph(graph_name, questions, boolean):
    
    image = f'{graph_name}.PNG'
    
  # Ask the user a question in multiple choice format
    
    display_grid(image, clear=boolean)
    # Ask the user questions about the grid
    panel = widgets.HBox([btn1, btn2, btn3])
    
    question_num = 0
    score = 0
    for question in questions:
        question_num += 1
        print(f"Q{question_num}/{len(questions)}")
        html_out = HTML(f"<span>{question}</span>")
        display(html_out ,panel)
        wait_for_event()
        answer = event_info["description"]
        data_dict["user_answer"].append(answer)
        if answer == questions[question]:
            score += 1
        clear_output()
        if boolean == False:
            display_grid(image)
    return score, len(questions)




In [10]:
data_dict= {"ID": [],
            "duration": [],
            "n_correct_answers": [],
            "hour_sleep": [],
            "gender": [],
            "user_answer": []}
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdIXdOiQSkp4EdwuNLYw7547Wtg5NVYTpvWh6YQPNYvyGstqw/viewform?usp=sf_link"

def main():

    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    We wish to record your response data
    to an anonymised public data repository. 
    Your data will be used for educational teaching purposes
    practising data analysis and visualisation.

    Please type   yes   in the box below if you consent to the upload."""

    print(data_consent_info)
    
    text_input = input("")
    if text_input == "yes":
        #show introduction
        display_intro(sentences, durations)
        #timing start
        start_time = time.time()
        # Analyze the first graph
        score1, len1 = analyze_graph('Picture1', q1, True)

        clear_output(wait=True)
        #question with the picture
        score2, len2 = analyze_graph('Picture2', q2, False)

        clear_output(wait=True)

        # Analyze the second graph
        score3, len3 = analyze_graph('Picture3', q3, True)

        clear_output(wait=True)

        score4, len4 = analyze_graph('Picture4', q4, False)

        clear_output(wait=True)

        score5, len5 = analyze_graph('Picture5', q5, True)

        clear_output(wait=True)

        score6, len6 = analyze_graph('Picture6', q6, False)

        clear_output(wait=True)
        #timing ends
        end_time = time.time()

        #calculate time taken
        time_taken = end_time - start_time
        rounded_time = round(time_taken, 2)
        data_dict["duration"] = rounded_time

        total_score = score1 + score2 + score3 + score4 + score5 + score6
        data_dict["n_correct_answers"] = total_score
        total_questions = len1 + len2 + len3 + len4 + len5 + len6

        conclusion_1 = (f"You got {total_score}/{total_questions} correct.\n",
                      f"You used {rounded_time} seconds to complete the test. Very good job!\n",
                      "End of the game! Thanks for your participation.")

        html_output = HTML(f"""<h2>{conclusion_1[0]}</h2>
                           <h2>{conclusion_1[1]}</h2>
                           <h2>{conclusion_1[2]}</h2>""")
        display(html_output)

        send_to_google_form(data_dict, form_url)
    else: 
        html_output = HTML("<h2>Thanks for your participation. Hope you have a nice day!</h2>")
        display(html_output)

In [11]:
main()

In [12]:
#to do list:

#add comments
#add docstrings
#Allows user to view their performance in comparison to the class dataset results